<a href="https://colab.research.google.com/github/onikesh/gscompany/blob/main/docs/start/start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2022 The Cirq Developers

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hello Qubit

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/start/start"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/main/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/start/start.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [2]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq

    print("installed cirq.")

installing cirq...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.0 MB/s eta 0:00:00
installed cirq.


In [112]:
import cirq

# 1. Definimos dois qubits
q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(3, 2)

# 2. Criamos o circuito de emaranhamento
circuit = cirq.Circuit()

# Passo A: Coloca o q0 em superposição (Porta Hadamard)
circuit.append(cirq.H(q0))

# Passo B: Aplica um CNOT (Controlled-NOT)
# Se q0 for 1, ele inverte o q1. Isso "prende" um ao outro.
circuit.append(cirq.CNOT(q0, q1))

# Passo C: Medimos ambos os qubits
circuit.append(cirq.measure(q0, q1, key='resultado'))

# 3. Simulamos o circuito 100 vezes
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1000)

# 4. Mostramos o circuito e os resultados
print("Circuito de Emaranhamento (Estado de Bell):")
print(circuit)
print("\nResultados (Repetições):")
print(result.histogram(key='resultado'))

print("\nNota: Repare que quase 100% dos resultados são '0' (que é 00) ou '3' (que é 11).")
print("É quase impossível sair '01' ou '10'. Os qubits estão sincronizados!")
print("_____________________________________________________________________________________")


# 1. Criamos 3 qubits: a mensagem, o canal e o destino
msg = cirq.NamedQubit("Mensagem")
alice_ref = cirq.NamedQubit("Alice (Canal)")
bob_ref = cirq.NamedQubit("Bob (Destino)")

circuit = cirq.Circuit()

# PASSO 1: Preparar a mensagem secreta (vamos usar a porta X para virar 1)
# Você pode trocar por qualquer porta, o Bob receberá o que você colocar aqui!
circuit.append(cirq.X(msg))

# PASSO 2: Criar o emaranhamento entre Alice e Bob (o "canal")
circuit.append([cirq.H(alice_ref), cirq.CNOT(alice_ref, bob_ref)])

# PASSO 3: Alice faz o protocolo de teletransporte
circuit.append([cirq.CNOT(msg, alice_ref), cirq.H(msg)])

# PASSO 4: Alice mede seus qubits e envia os resultados (clássicos) para Bob
circuit.append(cirq.measure(msg, alice_ref, key='medicao_alice'))

# PASSO 5: Bob ajusta o qubit dele com base no que a Alice mediu
# Se a medição da Alice indicar necessidade, Bob aplica portas X ou Z
circuit.append([cirq.CNOT(alice_ref, bob_ref), cirq.CZ(msg, bob_ref)])

# PASSO 6: Medimos o qubit do Bob para ver se a mensagem chegou!
circuit.append(cirq.measure(bob_ref, key='resultado_bob'))

# Simulação
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=100)

print("Circuito de Teletransporte:")
print(circuit)
print("\nResultado no Bob (deve ser tudo 1, pois teletransportamos a porta X):")
print(result.histogram(key='resultado_bob'))
print("_____________________________________________________________________________")


# 1. Configuração dos Qubits
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit()

# 2. Inicialização: Colocamos todos os estados em superposição
circuit.append(cirq.H.on_each(q0, q1))

# 3. O Oráculo: "Marca" a resposta correta (estamos procurando o 11)
# A porta CZ inverte o sinal apenas se ambos forem 1
circuit.append(cirq.CZ(q0, q1))

# 4. O Difusor: Amplifica a probabilidade da resposta marcada
circuit.append([cirq.H(q0), cirq.H(q1)])
circuit.append([cirq.X(q0), cirq.X(q1)])
circuit.append(cirq.CZ(q0, q1))
circuit.append([cirq.X(q0), cirq.X(q1)])
circuit.append([cirq.H(q0), cirq.H(q1)])

# 5. Medição
circuit.append(cirq.measure(q0, q1, key='resultado'))

# Simulação
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=100)

print("Algoritmo de Grover (Procurando '11'):")
print(result.histogram(key='resultado'))
# O resultado 3 em decimal é 11 em binário.



Circuito de Emaranhamento (Estado de Bell):
(0, 0): ───H───@───M('resultado')───
               │   │
(3, 2): ───────X───M────────────────

Resultados (Repetições):
Counter({3: 515, 0: 485})

Nota: Repare que quase 100% dos resultados são '0' (que é 00) ou '3' (que é 11).
É quase impossível sair '01' ou '10'. Os qubits estão sincronizados!
_____________________________________________________________________________________
Circuito de Teletransporte:
Alice (Canal): ───H───@───X───────M────────────────────@────────────────────────────
                      │   │       │                    │
Bob (Destino): ───────X───┼───────┼────────────────────X───@───M('resultado_bob')───
                          │       │                        │
Mensagem: ────────X───────@───H───M('medicao_alice')───────@────────────────────────

Resultado no Bob (deve ser tudo 1, pois teletransportamos a porta X):
Counter({1: 100})
_____________________________________________________________________________
Algo

In [113]:
!pip install qiskit qiskit-aer matplotlib pylatexenc --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.6 MB/s eta 0:00:00


In [121]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

# Criar circuito com 2 qubits e 2 bits clássicos
qc_bell = QuantumCircuit(2)

qc_bell.h(0)           # Superposição no Qubit 0
qc_bell.cx(0, 1)       # CNOT (Emaranhamento)
qc_bell.measure_all()  # Medição

# Simulação
sim = AerSimulator()
result = sim.run(qc_bell, shots=1000).result()
counts = result.get_counts()

print("--- Estado de Bell ---")
print(qc_bell.draw(output='text'))
print("\nResultados:", counts)


--- Estado de Bell ---
        ┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1 

Resultados: {'00': 513, '11': 487}


In [122]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

# Circuit com 3 qubits e 3 bits clássicos
# q0: Mensagem, q1: Alice, q2: Bob
qc_teleport = QuantumCircuit(3, 3)

# 1. Preparar a mensagem (Estado 1)
qc_teleport.x(0)
qc_teleport.barrier()

# 2. Criar canal de emaranhamento entre Alice(1) e Bob(2)
qc_teleport.h(1)
qc_teleport.cx(1, 2)
qc_teleport.barrier()

# 3. Protocolo da Alice
qc_teleport.cx(0, 1)
qc_teleport.h(0)
qc_teleport.barrier()

# 4. Medição da Alice
qc_teleport.measure([0, 1], [0, 1])
qc_teleport.barrier()

# 5. Correção do Bob (Portas condicionais)
qc_teleport.cx(1, 2)
qc_teleport.cz(0, 2)

# 6. Medição final do Bob para conferir se recebeu o '1'
qc_teleport.measure(2, 2)

# Simulação
sim = AerSimulator()
result = sim.run(qc_teleport, shots=10).result()
counts = result.get_counts()

print("--- Teletransporte Quântico ---")
print(qc_teleport.draw(output='text'))
# No Qiskit, o bit mais à esquerda no resultado representa o bit clássico de maior índice (Bob)
print("\nResultados (O primeiro dígito da esquerda deve ser 1):", counts)


--- Teletransporte Quântico ---
     ┌───┐ ░            ░      ┌───┐ ░ ┌─┐    ░            
q_0: ┤ X ├─░────────────░───■──┤ H ├─░─┤M├────░───────■────
     └───┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░       │    
q_1: ──────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░───■───┼────
           ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░ ┌─┴─┐ │ ┌─┐
q_2: ──────░──────┤ X ├─░────────────░──╫──╫──░─┤ X ├─■─┤M├
           ░      └───┘ ░            ░  ║  ║  ░ └───┘   └╥┘
c: 3/═══════════════════════════════════╩══╩═════════════╩═
                                        0  1             2 

Resultados (O primeiro dígito da esquerda deve ser 1): {'111': 3, '110': 1, '100': 2, '101': 4}


In [123]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

qc_grover = QuantumCircuit(2)

# 1. Inicialização (Superposição)
qc_grover.h([0, 1])

# 2. Oráculo (Marca o estado '11' invertendo a fase)
qc_grover.cz(0, 1)

# 3. Difusor (Amplifica a probabilidade da resposta correta)
qc_grover.h([0, 1])
qc_grover.x([0, 1])
qc_grover.cz(0, 1)
qc_grover.x([0, 1])
qc_grover.h([0, 1])

# 4. Medição
qc_grover.measure_all()

# Simulação
sim = AerSimulator()
result = sim.run(qc_grover, shots=100).result()
counts = result.get_counts()

print("--- Algoritmo de Grover ---")
print(qc_grover.draw(output='text'))
print("\nResultado da busca (deve ser '11' em 100% das vezes):", counts)


--- Algoritmo de Grover ---
        ┌───┐   ┌───┐┌───┐   ┌───┐┌───┐ ░ ┌─┐   
   q_0: ┤ H ├─■─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░─┤M├───
        ├───┤ │ ├───┤├───┤ │ ├───┤├───┤ ░ └╥┘┌─┐
   q_1: ┤ H ├─■─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░──╫─┤M├
        └───┘   └───┘└───┘   └───┘└───┘ ░  ║ └╥┘
meas: 2/═══════════════════════════════════╩══╩═
                                           0  1 

Resultado da busca (deve ser '11' em 100% das vezes): {'11': 100}


# Congratulations
You've just run your first Cirq program.

To learn about running a circuit on a virtual machine that mimics existing quantum hardware, see [Quantum Virtual Machine](../simulate/quantum_virtual_machine.ipynb).

If you would like to learn more about quantum computing, check out our [education page](https://quantumai.google/resources). The Full API reference for Cirq can be found [here](/reference/python/cirq). If you are looking for vendor specific information that can be found on our vendor sub-pages:


  [Alpine Quantum Technologies](../hardware/aqt/getting_started.ipynb)
  
  [Pasqal](../hardware/pasqal/getting_started.ipynb)
  
  [IonQ](../hardware/ionq/getting_started.ipynb)
  
  [Azure](../hardware/azure-quantum/getting_started_honeywell.ipynb)